Use [RISE](https://github.com/damianavila/RISE) to display the slides.

```bash
pip install RISE
```

<H1 style="text-align: center;">
Tekstitöötluse põhietapid
</H1>

<H4 style="text-align: center;">
Dage Särg
</H4>

<H4 style="text-align: center;">
Automaatne info eraldamine eestikeelsest tekstist. 27.10.2020
</H4>

<H4 style="text-align: center;">
Notebook colabis: TODO https://tinyurl.com/nlp-notebook
</H4>

<img src="pildid/põhietapid.png">

<img src="pildid/seg2.png">

## 1. Teksti segmenteerimine

In [ ]:
from estnltk import Text
# Teksti töötlemiseks peame tegema stringi Text objektiks
text = Text("Festivalil osales üle 30 000 muusikahuvilise.")
text

In [ ]:
# tag_layer() meetod märgib peale standardsed analüüsikihid,
# mida on vaja pea kõigi keeletöötlusülesannete juures
text.tag_layer()

In [ ]:
text.text

In [ ]:
# Tokens e sõned - mitte alati lingvistiliselt motiveeritud
text.tokens

In [ ]:
# Words e sõnad - mõned tokenid ühendatakse edasiseks töötluseks kokku
text.words

<img src="pildid/seg3.png">

In [ ]:
# On võimalik märgendada ka osalauseid
text = Text('Nendel, kes minu ja Oudekki kaotusele loodavad, \
on ettekujutus, et rahval polegi hääli.')

text.tag_layer(['clauses'])
text.clauses

<img src="pildid/seg4.png">

In [ ]:
# Laused - punkt ei toimi alati eraldajana
text = Text('Tartu Rattaralli toimub 29. mail 2020. \
Tartu Rattaralli stardi- ja finišipaik \
on traditsiooniliselt Tartu kesklinnas, Turu tänaval.')
text.tag_layer()
text.sentences

In [ ]:
# Tekst koosneb lausetest, mis koosnevad omakorda sõnadest
for sentence in text.sentences:
    print(' Lause: ', sentence.enclosing_text)
    for word in sentence:
        # Väljastame sõna ja sõnaliigi
        print( word.text, word.morph_analysis.partofspeech[0] )
    print()

## 2. Lemmatiseerimine, morfoloogiline analüüs & ühestamine

<img src="pildid/oad.png">

In [ ]:
# Morf analüüs eesti keele puhul baassamm 
t = Text('Mida ubadest teha? Oad võib salatisse panna.').tag_layer()
t.lemma

<img src="pildid/k2rbes.png">

<img src="pildid/morf1.png">

<img src="pildid/morf2.png">

In [ ]:
# Morf analüüs eesti keele puhul baassamm 
t = Text('Mida ubadest teha? Oad võib salatisse panna.').tag_layer()
t.morph_analysis

In [ ]:
# Võime vaadata ka vaid parajasti huvitavaid atribuute, mitte kogu analüüsi
t.morph_analysis['text', 'lemma', 'partofspeech']

#### Näide: leiame kõik tekstis olevad nimisõnad

In [ ]:
my_text = Text('Nagu nimigi reedab, on nurgasaag kõige \
tõhusam tööriist erinevate puitdetailide lõikamiseks, \
kus eesmärgiks on saavutada täpne lõikenurk ning oluline on \
lõikenurga seadistamise võimalus. Näiteks pildiraamide \
meisterdamisel, kus on oluline, et detailide lõikenurgad \
oleksid kõik täpselt 45 kraadi. Sellisel juhul on nurgasaag \
täiuslikuks tööriistaks, sest tagab täpsuse ja lõike korratavuse. \
Üldiselt on valdav osa nurgasaage seadistatavad 45-kraadise \
lõikenurga alla vähemalt ühes suunas. Lisaks võimaldavad mõned \
saed veel ka saetera kaldenurga seadistamist, mis tuleb kasuks \
keerukamate detailide lõikamisel. Nurgasaag on väga tõhus ka \
kitsamate, kuni 30 cm laiuste puulaudade või muude puitdetailide \
ristlõigete tegemiseks ehk järkamiseks, mida tuleb palju ette näiteks \
puitkonstruktsioonide ehitamisel või ka näiteks terrassilaudade \
või puitparketi paigaldamisel.')
my_text.tag_layer()

In [ ]:
noun_lemmas = []
for lemmas, postags in zip(my_text.lemma, my_text.partofspeech):
    if 'S' in postags: # text.lemma ja partofspeech on listid, kuna analüüse võib olla mitu
        noun_lemmas += lemmas
noun_lemmas  

from collections import Counter
Counter(noun_lemmas).most_common()

### Ülesanne 1. Leidke eelnevast tekstist kõik da-vormis verbi sisaldavad laused

#### Ülesande 1 lahendus: leiame kõik infinitiivset verbi sisaldavad laused

In [ ]:
infinitive_sentences = []
for sent in my_text.sentences: # vaatame teksti lause kaupa
    for form in sent.form: # vaatame läbi kõik lause sõnade vormiinfod
        if 'da' in form:
            a = sent.enclosing_text # lause tekst stringina
            infinitive_sentences.append(a)
            break
infinitive_sentences

### 2a. Morfoloogilise analüüsi parameetrite muutmine

### Morfoloogiline analüsaator vaikimisi:
#### 1. Leiab reegli+sõnastikupõhiselt kõik võimalikud analüüsivariandid kõigile sõnadele
#### 2. Oletab analüüsivariandid sõnadele, mida sõnastiku abil analüüsida ei saanud
#### 3. Teostab ühestamise - eemaldab konteksti sobimatud variandid

In [ ]:
# Näide koos oletamisega - analüüsitakse ära kõik "sõnad"
t = Text("Naiste teksapüksid Guess Jeans W 29/32 F0607421_Hali").tag_layer()
t.morph_analysis

In [ ]:
# Morfanalüsaatori parameetrite muutmiseks peame kasutama resolverit
from estnltk.resolve_layer_dag import make_resolver

resolver = make_resolver(
                 disambiguate=False, # ilma oletamiseta ei saa teostada ka ühestamist
                 guess=False, # keelame oletamise
                 propername=False, # ilma oletamiseta ei saa teostada ka pärisnimeanalüüsi
                 phonetic=False,
                 compound=True)

In [ ]:
# Näide ilma oletamiseta - analüüsitakse vaid päris sõnad, numbrid, lühendid
t = Text("Naiste teksapüksid Guess Jeans W 29/32 F0607421_Hali").tag_layer(resolver = resolver)
t.morph_analysis

In [ ]:
# Ühestajal ei ole alati õigus - vahel tahaks kustutatud variante ka
t = Text('Mees oli kärbes.').tag_layer()
t.morph_analysis

In [ ]:
# Morf analüüs ilma ühestamiseta - saame kätte ka õige variandi 
text = Text("Mees oli kärbes.")
text.tag_layer(resolver=resolver)['morph_analysis']

### 2b. Korpuspõhine morfoloogiline analüüs & ühestamine

#### Morfoloogiline analüsaator vaikimisi arvestab ühestamisel kontekstina üht lauset
#### Tegelikkus:
 * tihti kehtib põhimõte "üks tekst, üks tähendus"
 * laiemat konteksti arvestades esinevad sõnad eri vormides ning selle põhjal saab kätte algvormi

In [ ]:
# Näide - lausepõhine ühestamine
corpus_texts = ['Esimesele kohale tuleb Jänes, kuigi tema \ 
                punktide summa pole kõrgeim.',
                'Lõpparvestuses läks Konnale esimene koht. Teise koha sai \
                seekord Jänes. Uus võistlus toimub 2. mail.', \
                'Konn paistis silma suurima punktide summaga. \ 
                Uue võistluse toimumisajaks on 2. mai.']

In [ ]:
corpus = [Text(t).tag_layer(['morph_analysis']) for t in corpus_texts]
corpus

In [ ]:
for text in corpus:
    print(text.text)
    # Trükime välja mitmeseks jäänud sõnade analüüsid 
    # (ainult lemma, sõnaliigi ja vorminimetuse)
    for word in text.morph_analysis:
        if len(word.annotations) > 1:
            for a in word.annotations:
                print(word.text, '=>',(a.lemma, a.partofspeech, a.form))
    print()

In [ ]:
# Korpuspõhine ühestamine
from estnltk.taggers import VabamorfCorpusTagger

# Loome tagger'i ning määrame selle lisatava kihi:
vm_corpus_tagger = VabamorfCorpusTagger(output_layer='morph_with_cb_disamb')

vm_corpus_tagger.tag(corpus)

In [ ]:
for text in corpus:
    print(text.text)
    # Trükime välja mitmeseks jäänud sõnade analüüsid 
    # (ainult lemma, sõnaliigi ja vorminimetuse)
    for word in text.morph_with_cb_disamb:
        if len(word.annotations) > 1:
            for a in word.annotations:
                print(word.text, '=>',(a.lemma, a.partofspeech, a.form))
    print()

### 2c. Morfoloogiline analüüs kasutajasõnastiku abil

#### Kui teame, et meie tekstis esinevad püsivalt mingid kirjakeelest erinevad sõnad/vormid, saame nende analüüsi kehtestada kasutajasõnastiku abil.

In [ ]:
# Tavapärasel morf analüüsil saab sõna "pand" nimisõna analüüsi ning "talu" tegusõna
t = Text("ma küll ei pand talu põlema").tag_layer()
t.morph_analysis

<img src="pildid/pand.png">

In [ ]:
t = Text("ma küll ei pand talu põlema").tag_layer(resolver=resolver)
t.morph_analysis

In [ ]:
from estnltk.taggers import UserDictTagger
userdict = UserDictTagger(ignore_case=True)  # vaikimisi suurtähetundlik
# Lisame oma tundmatu sõna kasutajasõnastikku
userdict.add_word('pand', {'root': 'pane', 'partofspeech': 'V', 'form': 'nud', 'ending': 'nud'})
userdict.retag(t)
t.morph_analysis

In [ ]:
# Teostame nüüd ühestamise korrektsetel analüüsidel
from estnltk.taggers import VabamorfDisambiguator

vm_disambiguator = VabamorfDisambiguator()
vm_disambiguator.retag(t)
t.morph_analysis

### 4. Süntaktiline analüüs

<img src="pildid/synt1.png">

<img src="pildid/synt2.png">

<img src="pildid/synt3.png">

<img src="pildid/synt4.png">

<img src="pildid/synt5.png">

<img src="pildid/synt6.png">

## Märgendajad e taggerid
#### võimaldavad vajadusel lisada kihte, mida läheb vaja mingis konkreetses analüüsitöövoos

In [ ]:
# süntaksi tagger
from estnltk.taggers import MaltParserTagger
maltparser_tagger = MaltParserTagger()
maltparser_tagger

In [ ]:
from estnltk.taggers import ConllMorphTagger
conllmorph_tagger = ConllMorphTagger()
conllmorph_tagger

In [ ]:
# Lisame default_layers kihid
text = Text('Poe ees õlut joonud mehed häirisid kohalikke, aga ükskõikne \ 
politsei ei teinud midagi.').tag_layer(['sentences', 'words', 'morph_extended'])

# Lisame conll_morph kihi
# Ei tööta colab'is ega teie arvutis, kui te pole installinud VislCG3
# Võimalik installida siit https://visl.sdu.dk/cg3/chunked/installation.html,
# aga plaanis on sellest sõltuvusest vabaneda. Hetkel on süntaksi töövoog
# EstNLTK-s veel väga savijalgadel.
conllmorph_tagger.tag(text)

In [ ]:
# Lõpuks MaltParseri kiht
# See ilma VislCG3-ta samuti ei tööta
maltparser_tagger.tag(text)
text.maltparser_syntax

## 5. Semantiline analüüs

### Ajaväljendite tuvastamine

In [ ]:
text3 = Text('EKA sisearhitektuuri osakond ja RMK \
avavad neljapäeval kell 16.00 \
RMK Tallinna kontoris (Toompuiestee 24) näituse')

In [ ]:
# Märgendame ajaväljendid
from estnltk.taggers import TimexTagger

tagger = TimexTagger()
text3.tag_layer()
tagger.tag( text3 )

In [ ]:
text3.timexes

In [ ]:
# Soovi korral võime määratleda teksti loomise aja
text3 = Text('EKA sisearhitektuuri osakond ja RMK \
avavad neljapäeval kell 16.00 \
RMK Tallinna kontoris (Toompuiestee 24) näituse').tag_layer()
text3.meta['document_creation_time'] = '2019-10-27'
tagger.tag(text3)

In [ ]:
text3.timexes

### Aadresside tuvastamine

In [ ]:
# Toimub kahes etapis
from estnltk.taggers import AddressPartTagger, AddressGrammarTagger
address_token_tagger = AddressPartTagger(output_layer='address_tokens')
address_tagger = AddressGrammarTagger(output_layer='addresses', 
                                      input_layer='address_tokens')

In [ ]:
text = Text("Ootame teid 2. novembril külla \
aadressil Aia 6, Tartu.").tag_layer(['words'])

In [ ]:
# Esiteks märgime peale võimalikud aadresside komponendid
address_token_tagger.tag(text)["address_tokens"]

In [ ]:
# Teiseks leiame aadressid sealt, kus sobivad komponendid järjest esinevad
address_tagger.tag(text)['addresses']

<img src="pildid/verb.png">

In [ ]:
from estnltk.taggers import VerbChainDetector
vc_detector = VerbChainDetector()

In [ ]:
from estnltk import Text
# Loome teksti
text = Text('Kas Juku alustas kodutööga? Minuteada ei alustanud.')
# Lisame verbiahelate tuvastamiseks vajalikud sisendkihid
text.tag_layer(['words', 'sentences', 'morph_analysis', 'clauses'])

In [ ]:
# Tuvastame verbiahelad
vc_detector.tag( text )

# Väljastame verbiahelad (vastavad tekstid)
text.verb_chains

## 4. Oma märgendajate kirjutamine

### PhraseTagger
#### võimaldab märgendada kihis järjest esinevaid elemente mingi atribuudi alusel

Proovime kirjutada taggerit, mis märgendaks lihtsaid nimisõnafraase.

In [ ]:
from estnltk.taggers import PhraseTagger

# Kasutame fraaside esmaseks määratlemiseks sõnaliike
phrase_list = [
               { '_phrase_': ('A', 'S')},
               { '_phrase_':  ('C', 'S')},
               { '_phrase_':  ('U', 'S')}
              ]

In [ ]:
# Defineerime taggeri, mis phrase_list muutujas olevaid fraasitüüpe märgendaks
phrase_tagger = PhraseTagger(output_layer='noun_phrases',
                      input_layer='morph_analysis',
                      input_attribute='partofspeech',
                      vocabulary=phrase_list,
                      key='_phrase_')

In [ ]:
# Rakendame kirjutatud taggerit morfanalüüsitud tekstile
t = Text('Viimasedki pardid lendasid soojemale maale, \
kui jää läks liiga paksuks jõest toidu hankimiseks.').tag_layer()
phrase_tagger.tag(t)

In [ ]:
# Leiame nimisõnafraasid
# Puuduvad algvormid
# Paremate tulemuste jaoks peaks arvesse võtma rohkem kui sõnaliike
t.noun_phrases

In [ ]:
# Dekoraator võimaldab lisada oma uuele kihile atribuute - lisame lemmad
def decorator(span, annotation):
    annotation['lemmas'] = ' '.join([l[0] for l in span.lemma])
    return True

In [ ]:
# Uus phrase_tagger, mis paneb uude kihti ka fraasid algvormis
phrase_tagger2 = PhraseTagger(output_layer='noun_phrases2',
                      input_layer='morph_analysis',
                      input_attribute='partofspeech',
                      vocabulary=phrase_list,
                      key='_phrase_',
                      output_attributes = ['lemmas'],
                      decorator = decorator)

In [ ]:
# Olemas ilusad algvormis fraasid
# Vaja oleks ka sõna vormiinfot arvestada
phrase_tagger2.tag(t)
t.noun_phrases2

In [ ]:
# Täiendame dekoraatorit - arvestame ka vormide ühilduvust
def decorator2(span, annotation):
    annotation['lemmas'] = ' '.join([l[0] for l in span.lemma])
    
    ninataga_sg = ['sg ter', 'sg ab', 'sg kom', 'sg es']
    ninataga_pl = ['pl ter', 'pl ab', 'pl kom', 'pl es']
    # Omadussõna ja nimisõna samas vormis -> OK
    if span[0].form == span[1].form:
        return True
    # Omadussõna ainsuse omastavas ja nimisõna 4 viimases käändes ainsuses -> OK
    elif span[0].form[0] == 'sg g' and span[1].form[0] in ninataga_sg:
        return True
    # Omadussõna mitm omastavas ja nimisõna 4 viimases käändes mitm -> OK
    elif span[0].form[0] == 'pl g' and span[1].form[0] in ninataga_pl:
        return True
    # Kõik muud juhud -> ei sobi fraas
    else: 
        return False

In [ ]:
phrase_tagger3 = PhraseTagger(output_layer='noun_phrases3',
                      input_layer='morph_analysis',
                      input_attribute='partofspeech',
                      vocabulary=phrase_list,
                      key='_phrase_',
                      output_attributes = ['lemmas'],
                      decorator = decorator2)

In [ ]:
# Märgendame kolmanda nimisõnafraaside kihi
phrase_tagger3.tag(t)

In [ ]:
# Tulemused vastavad ootustele
 t.noun_phrases3

In [ ]:
# Fraase on lihtne stringidena kätte saada
for i in t.noun_phrases3:
    print(i.lemmas)

### Ülesanne 2. Leidke failist data/proverbs.txt, millised nimisõnafraasid esinevad kõige sagedamini eesti vanasõnades

#### Ülesande 2 lahendus: kõige sagedasemad nimisõnafraasid eesti vanasõnades

In [ ]:
with open("data/proverbs.txt", "r", encoding = 'utf8') as fin:
    # Failis on iga vanasõna eraldi real - saame listi vanasõnadest
    proverbs = fin.readlines()

In [ ]:
from collections import Counter
from tqdm import tqdm

# Loendur fraaside kokkulugemiseks
noun_phrases_counts = Counter()

for text in tqdm(proverbs): # vaatame vanasõnade listi järjest läbi
    t = Text(text).tag_layer() # teeme vanasõna Text objektiks ja analüüsime
    phrase_tagger3.tag(t) # märgime peale nimisõnafraasid oma parima taggeriga
    if len(t.noun_phrases3) > 0: 
        for p in t.noun_phrases3: # suurendame loendurit vastava fraasi kohal
            noun_phrases_counts[p.lemmas] += 1

In [ ]:
# Saamegi kätte sagedasemad nimisõnafraasid
noun_phrases_counts.most_common(10)

### Nimeüksuste tuvastamine (NER - Named Entity Recognition)

EstNLTK sisaldab automaatset nimeüksuste tuvastajat. 

Programm võimaldab tuvastada 3 liiki nimeüksuseid:

* isikunimesid ( lühend: PER );

* asukohanimesid ( lühend: LOC );

* organisatsiooninimesid ( lühend: ORG )

In [ ]:
from estnltk.taggers import NerTagger
ner_tagger = NerTagger()

# Milliseid kihte ner_tagger vajab?
ner_tagger.input_layers

In [ ]:
from estnltk import Text
# tekitame näidisteksti
t = Text(''' Eesti President on Kersti Kaljulaid. Eesti Energia on \ 
Eesti riigile kuuluv rahvusvaheline energiaettevõte. ''')

# lisame tekstile vajamineva 'morph_analysis' kihi
t.tag_layer('morph_analysis')

# lisame nimeüksuste märgenduse
ner_tagger.tag(t)

In [ ]:
# väljastab tuvastatud nimeüksused
t.ner

### Oma nimeüksuste lisamine

1) Leiame võimalikult suure hulga näiteid vastavast nimeüksusest
 
2) Märgendame ümber olemasoleva korpuse - anname seal esinevatele vastavatele nimeüksustele soovitud märgendid

3) Treenime nimeüksuste tuvastaja ümbermärgendatud korpuse peal

4) Märgendame oma nimeüksusi

### Oma nimeüksuste lisamine - kuidas luua leksikone?
1) Spetsiifilised allikad - nt eesnimede loetelu, ametite loetelu

2) Wordnet

3) Suurel korpusel treenitud sõnavektorid - nt word2vec


<img src="pildid/baas1.png">

<img src="pildid/baas2.png">

<img src="pildid/baas3.png">

<img src="pildid/baas7.png">

<img src="pildid/baas4.png">

In [ ]:
# Wordneti kasutamiseks tuleb importida vastav moodul ja luua Wordneti objekt:
from estnltk.wordnet import Wordnet

wn = Wordnet()
toit = wn['toit']
toit

In [ ]:
toit[0].definition

In [ ]:
toit[2].definition

In [ ]:
toidud = toit[2].closure('hyponym')

In [ ]:
toidud[:20]

In [ ]:
len(toidud)

In [ ]:
toidud[1000].lemmas

<img src="pildid/välised.png">

<img src="pildid/ressursid1.png">

### Ülesanne 3. Kes on teksti peategelane?

Koolituse materjalide hulgas on toodud üks artikkel portaalist Telegram - fail nimega 'data/telegram.txt'. Leidke, millist olendit on kõige enam selles tekstis mainitud. 

#### Ülesande 3 täpsustus:
* leidke tekstist kõik nimisõnade lemmad
* leidke wordnetist kõik olendeid tähistavad sõnad. Kasutage sünohulka wn.synset("olend.n.01") - leidke selle closure() ehk kõik alamsünohulgad ning omakorda nende hulkade lemmad
* kontrollige, millised tekstis esinenud nimisõnadest kuuluvad olendite hulka
* leidke nendest sagedasim

#### Ülesande 3 lahendus: leiame teksti peategelase

In [ ]:
from collections import Counter
import pandas as pd

# Kirjutame olendid faili, kuna nende leidmine võtab pisut aega (~5 min)
with open('data/olendid.txt', 'w', encoding='utf-8') as f:
    olend_synsets = wn['olend'][0].closure('hyponym')
    for olend_synset in tqdm(olend_synsets):
        for lemma in olend_synset.lemmas:
            f.write(lemma + '\n')

In [ ]:
with open('data/telegram.txt', 'r', encoding='utf-8') as f:
    text = Text(f.read())

text.tag_layer()

text_substantives = list()

for analysis in text.morph_analysis:
    pos_lemmas = zip(analysis.partofspeech, analysis.lemma)
    for pos, lemma in pos_lemmas:
        if pos == 'S':
            text_substantives.append(lemma)

text_substantives = Counter(text_substantives)

In [ ]:
olend_lemmas = list()

with open('data/olendid.txt', 'r', encoding='utf-8') as fin:
    for line in fin.readlines():
        olend_lemmas.append(line.strip())

telegram_olendid = dict()

for word, wordcount in text_substantives.items():
    if word in olend_lemmas:
        telegram_olendid[word] = wordcount
        
telegram_olendid = Counter(telegram_olendid)
telegram_olendid.most_common(1)

<img src="pildid/masin1.png">